In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import tensorflow as tf
from tensorflow.keras import utils,Sequential,layers


2025-04-19 23:23:22.310570: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-04-19 23:23:22.315969: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-04-19 23:23:22.336551: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745105002.396010    6232 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745105002.412014    6232 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1745105002.445883    6232 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linkin

### 1.Importação de bibliotecas

In [4]:
# Install the missing system library required by OpenCV
!apt-get update && apt-get install -y libgl1-mesa-glx

import cv2

im = cv2.imread('/kaggle/input/retina-blood-vessel/Data/train/image/1.png')
print(type(im))
# <class 'numpy.ndarray'>

print(im.shape)
print(type(im.shape))
# (512, 512, 3)
# <class 'tuple'>


Reading package lists... Done
E: List directory /var/lib/apt/lists/partial is missing. - Acquire (13: Permission denied)


ImportError: libGL.so.1: cannot open shared object file: No such file or directory

Since the Data is so large lets normalize and load it and also minimize it to 128,128,3


In [4]:
train= utils.image_dataset_from_directory('/kaggle/input/retina-blood-vessel/Data/train',image_size=(128,128),seed=42)
test= utils.image_dataset_from_directory('/kaggle/input/retina-blood-vessel/Data/test',image_size=(128,128),seed=42)
def process(image,label):
    image = tf.cast(image/255. ,tf.float32)
    return image,label

train = train.map(process)
test = test.map(process)


Found 160 files belonging to 2 classes.
Found 40 files belonging to 2 classes.


Since the dataset is realtivally small lets create our own model withou transfer learning

### Modelagem


1. **Extração de Características:** A imagem de entrada passa por uma série de blocos convolucionais (Conv2D, MaxPool2D, BatchNormalization). Cada bloco reduz a resolução espacial da imagem (MaxPool2D) enquanto aumenta a capacidade de detectar padrões complexos (Conv2D com diferentes números de filtros). A BatchNormalization ajuda a estabilizar o processo. A ideia é que as camadas iniciais detectem características simples (bordas, texturas) e as camadas mais profundas detectem características mais complexas e abstratas (partes de objetos, etc.).

2. **Achatamento:** Após a extração de características espaciais, a camada Flatten transforma tudo em um único vetor longo.

3. **Classificação:** Este vetor é então alimentado na camada Dense final. Como há apenas um neurônio com ativação sigmoid, o modelo produzirá um único valor entre 0 e 1, indicando a probabilidade de a imagem pertencer a uma determinada classe (assumindo um problema de classificação binária).


In [4]:
model = Sequential([
    layers.Conv2D(128,5,activation="relu",input_shape=(128,128,3)),
    layers.MaxPool2D(),
    layers.BatchNormalization(),
    
    layers.Conv2D(64,5,activation="relu"),
    layers.MaxPool2D(),
    layers.BatchNormalization(),

    layers.Conv2D(32,5,activation="relu"),
    layers.MaxPool2D(),
    layers.BatchNormalization(),

    layers.Conv2D(16,5,activation="relu"),
    layers.MaxPool2D(),
    layers.BatchNormalization(),
    
    layers.Flatten(),
    layers.Dense(1,activation="sigmoid")
])


In [5]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_4 (Conv2D)               │ (None, 124, 124, 128)  │         9,728 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 62, 62, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 62, 62, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 58, 58, 64)     │       204,864 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 29, 29, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (None, 29, 29, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 25, 25, 32)     │        51,232 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_6 (MaxPooling2D)  │ (None, 12, 12, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_6           │ (None, 12, 12, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 8, 8, 16)       │        12,816 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_7 (MaxPooling2D)  │ (None, 4, 4, 16)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_7           │ (None, 4, 4, 16)       │            64 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           257 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 279,857 (1.07 MB)

 Trainable params: 279,377 (1.07 MB)

 Non-trainable params: 480 (1.88 KB)

WOW 279k+ prams lets continue

lets compile the model with binary_crossentropy as the loss, Adam as optimizer and Accuracy and recall as metrics


In [6]:
model.compile(loss="binary_crossentropy",
              optimizer="Adam",
              metrics=["Accuracy"]
)

lets get some callbacks

In [7]:
es = tf.keras.callbacks.EarlyStopping(
    monitor='Accuracy',
    min_delta=0,
    patience=0,
    verbose=0,
    mode='auto',
    baseline=None,
    restore_best_weights=True,
    start_from_epoch=10
)


In [8]:
with tf.device('/GPU:0'):
    model.fit(train,epochs=50,callbacks=[es])

NameError: name 'train' is not defined

will that went 'to great' jokes asside everything was perfect great job we got a 100% on training great job!!